In [1]:
from iteru import *

In [2]:
from statistics import mode

In [3]:
from ipywidgets import *

In [4]:
aoi = GERD_aoi_dam

In [5]:
dates_dict = {'2020-07-09': 1594265117000,
 '2020-07-21': 1595301918000,
 '2020-08-02': 1596338718000,
 '2020-08-14': 1597375519000,
 '2020-08-26': 1598412320000,
 '2020-09-07': 1599449120000,
 '2020-09-19': 1600485921000,
 '2020-10-01': 1601522721000,
 '2020-10-13': 1602559521000,
 '2020-10-25': 1603596321000,
 '2020-11-06': 1604633121000,
 '2020-11-18': 1605669921000,
 '2020-11-30': 1606706720000,
 '2020-12-12': 1607743520000,
 '2021-01-05': 1609817119000,
 '2021-01-17': 1610853918000,
 '2021-01-29': 1611890718000,
 '2021-02-10': 1612927518000,
 '2021-02-22': 1613964317000,
 '2021-03-06': 1615001117000,
 '2021-03-18': 1616037917000,
 '2021-03-30': 1617074717000,
 '2021-04-11': 1618111518000,
 '2021-04-23': 1619148318000,
 '2021-05-05': 1620185119000,
 '2021-05-17': 1621221919000,
 '2021-05-29': 1622258720000,
 '2021-06-10': 1623295521000,
 '2021-06-22': 1624332322000,
 '2021-07-04': 1625369122000,
 '2021-07-16': 1626405923000,
 '2021-07-28': 1627442724000,
 '2021-08-09': 1628479525000,
 '2021-08-21': 1629516325000,
 '2021-09-02': 1630553126000,
 '2021-09-14': 1631589926000,
 '2021-09-26': 1632626727000,
 '2021-10-08': 1633663527000,
 '2021-10-20': 1634700327000,
 '2021-11-01': 1635737127000,
 '2021-11-13': 1636773927000,
 '2021-11-25': 1637810726000,
 '2021-12-07': 1638847526000,
 '2021-12-19': 1639884325000,
 '2021-12-31': 1640921125000,
 '2022-01-12': 1641957924000,
 '2022-01-24': 1642994724000,
 '2022-02-05': 1644031523000,
 '2022-03-01': 1646105123000,
 '2022-03-13': 1647141923000,
 '2022-03-25': 1648178723000,
 '2022-04-06': 1649215523000,
 '2022-04-18': 1650252324000,
 '2022-04-30': 1651289124000}

In [6]:
date_number = Dropdown(options=dates_dict)
date_number

Dropdown(options={'2020-07-09': 1594265117000, '2020-07-21': 1595301918000, '2020-08-02': 1596338718000, '2020…

In [187]:
SAR = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filter(ee.Filter.equals('relativeOrbitNumber_start', 50))\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.eq('resolution_meters',10))\
    .filterBounds(aoi)\
    .filter(ee.Filter.eq('system:time_start',date_number.value))\
    .select(['VV','VH'])\
    .median()

In [188]:
VV_smooth = ee.Image(toDB(RefinedLee(toNatural(SAR.select(['VV']))))).rename('VV_Filtered')
VH_smooth = ee.Image(toDB(RefinedLee(toNatural(SAR.select(['VH']))))).rename('VH_Filtered')

In [189]:
histogram_VV = VV_smooth.select('VV_Filtered').reduceRegion(               
            reducer = ee.Reducer.histogram().combine('mean', None, True).combine('variance', None, True),
            geometry = aoi, 
            scale = 10,
            bestEffort = True   )
histogram_VH = VH_smooth.select('VH_Filtered').reduceRegion(               
            reducer = ee.Reducer.histogram().combine('mean', None, True).combine('variance', None, True),
            geometry = aoi, 
            scale = 10,
            bestEffort = True   )

In [190]:
otsu_threshold_VV = otsu(histogram_VV.get('VV_Filtered_histogram'))
otsu_threshold_VH = otsu(histogram_VH.get('VH_Filtered_histogram'))

In [191]:
water_mask_VV = VV_smooth.select('VV_Filtered').lt(otsu_threshold_VV).selfMask().rename('water_mask')
water_mask_VH = VH_smooth.select('VH_Filtered').lt(otsu_threshold_VH).selfMask().rename('water_mask')

In [192]:
water_mask_VV = water_mask_VV.select('water_mask').clip(aoi)
water_mask_VH = water_mask_VH.select('water_mask').clip(aoi)

In [193]:
feature_VV = ee.Image(1).updateMask(water_mask_VV).reduceToVectors(
           geometry = water_mask_VV.geometry(),
           crs ='EPSG:32636',
           scale = 10 , 
           geometryType = 'polygon',
           eightConnected = False,
           labelProperty =  'water_cover',
           bestEffort = True
          )
feature_VH = ee.Image(1).updateMask(water_mask_VH).reduceToVectors(
           geometry = water_mask_VH.geometry(),
           crs ='EPSG:32636',
           scale = 10 , 
           geometryType = 'polygon',
           eightConnected = False,
           labelProperty =  'water_cover',
           bestEffort = True
          )

In [194]:
def calc_area(feature):
    
    area = feature.geometry().area(maxError = 1)
    
    return feature.set({'Area':area})

In [195]:
feature_VV = feature_VV.map(calc_area)
feature_VH = feature_VH.map(calc_area)

In [196]:
lake_feature_VV = feature_VV.sort('Area', False).first()
lake_feature_VH = feature_VH.sort('Area', False).first()

In [197]:
lake_area_VV = lake_feature_VV.geometry().area(maxError = 1) 
lake_area_VH = lake_feature_VH.geometry().area(maxError = 1) 

In [198]:
fabdem = ee.ImageCollection("projects/sat-io/open-datasets/FABDEM").median()

In [199]:
# lake_dem_VV = elevation_dataset.clip(lake_feature_VV)
# lake_dem_VH = elevation_dataset.clip(lake_feature_VH)

In [200]:
lake_dem_VV = fabdem.clip(lake_feature_VV)
lake_dem_VH = fabdem.clip(lake_feature_VH)

In [201]:
elevations_VV = lake_dem_VV.reduceRegion(
        reducer=ee.Reducer.toList(), 
        geometry=lake_dem_VV.geometry(),
        maxPixels=1e11,
        scale=30,
        crs = 'EPSG:32636',
        bestEffort = True
    ).get('b1')
elevations_VH = lake_dem_VH.reduceRegion(
        reducer=ee.Reducer.toList(), 
        geometry=lake_dem_VH.geometry(),
        maxPixels=1e11,
        scale=30,
        crs = 'EPSG:32636',
        bestEffort = True
    ).get('b1')

In [202]:
elevations_list_VV = ee.List(elevations_VV).getInfo()
elevations_list_VH = ee.List(elevations_VH).getInfo()

In [203]:
elevations_desc_VV = sorted(elevations_list_VV,reverse=True)
elevations_desc_VH = sorted(elevations_list_VH,reverse=True)

In [204]:
water_level_VV= mode(elevations_desc_VV[:1000])
water_level_VH= mode(elevations_desc_VH[:1000])

In [205]:
volume_VV = ((water_level_VV*len(elevations_desc_VV)-sum(elevations_desc_VV))*900)/1e9
volume_VH = ((water_level_VH*len(elevations_desc_VH)-sum(elevations_desc_VH))*900)/1e9

In [206]:
print('VV Band\n\n'
      f'{otsu_threshold_VV.getInfo():0.2f}, {lake_area_VV.getInfo()/1e6:0.2f}, {water_level_VV:0.2f}, {volume_VV:0.2f}'
       '\n'
      f'{otsu_threshold_VH.getInfo():0.2f}, {lake_area_VH.getInfo()/1e6:0.2f}, {water_level_VH:0.2f}, {volume_VH:0.2f} '
      )

VV Band

-15.87, 267.04, 578.12, 7.30
-22.62, 311.95, 605.76, 16.04 
